# Mode Explorer
## 1. Dependencies:
* numpy
* cython
* matplotlib
* pyqt4
* pyo  #python setup.py install at the pyo directory. Please use the local directory for the custom fifo player



In [8]:
# Uncomment it, it only need to be run once. 
# Depends on whether the popup window is showing. 
%pylab qt4 

Populating the interactive namespace from numpy and matplotlib


/Users/jiajunyang/anaconda3/envs/mode_explorer/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# %matplotlib qt
%load_ext Cython
import matplotlib.pyplot as plt
import numpy as np
import pyo
import math, time, random, threading
import logging
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


## 2. Description:
Based on Particle Trajectory Sonification. Sonify mode of high dimensional numerical data. Interact with GUI window.

## 3. Start a fifo player
This allows audio clips to be played on demand 'instantly'.

In [4]:
sr = 11025.
ser = pyo.Server(sr=int(sr), nchnls=1, buffersize=512)  # pyo server
ser.boot()
fifo = pyo.FIFOPlayer()
out = pyo.Biquad(fifo, freq=5000, q=1, type=0).out()  # BPF
ser.start()

## 4. Core Method: Particle Trajectory Sonification

In [5]:
%%cython -2
cimport numpy as np
import numpy as np
from libc.stdlib cimport rand, malloc, free
from libc.math cimport exp

def potential(np.ndarray[np.float64_t, ndim=2] data,\
              np.ndarray[np.float64_t, ndim=1] pos,\
              double sigma=0.2):
    """Potential energy function. 
    
    Args:
      data (float64): your data 
      
      pos (float64): current position of the particle
      
      sigma (double): sigma attribute

    Returns:
      potential double : potential energy of the particle
    """
    cdef int N, dim, j, i
    cdef double potential, distsq, h
    cdef double minusOneOverTwoSigmaSquared = -0.5/(sigma*sigma)
    N, dim = data.shape[0], data.shape[1]
    for j in range(N):
        distsq = 0
        for i in range(dim): 
            h = pos[i] - data[j, i]
            distsq += h*h
        potential += -exp(minusOneOverTwoSigmaSquared * distsq)
    return potential

# trj, sig, lastpos, lastvel = PTSM(data, initialpos, initialvel, sigma, mass, r, dt, nrSteps)
def PTSM(np.ndarray[np.float64_t, ndim=2] data,\
         np.ndarray[np.float64_t, ndim=1] initialpos,\
         np.ndarray[np.float64_t, ndim=1] initialvel,\
         double sigma=0.25, double mass=1,\
         double r=0.99, double dt=0.01, int nrSteps=1000):
    """Particle trajectory, calculate the potential energy 
    at each position in the data space
    and figure where to go next. 
    
    Args:
      data (np float64): your data 
      
      initialpos (np float64): initial position of the particle
      
      initialvel (np float64): initial velocity of the particle
      
      sigma (double): sigma attribute
      
      mass (double): mass, normally just use 1.
      
      r (double): resistence, the larger the quicker velocity decay, shorter sound
      
      dt (double) : this will be used to multiply with velocity for the distance change 
      
      nrSteps (int) : the amount of steps it take for each iteraction. The shorter the more precise movement. 

    Returns:
      trj (np.array): trajectory for plotting
      sig (np.array): audio signal for playback
      lastpos (np.array): the previous positions
      lastvel (np.array): the previous velocity
    
    """
    cdef int N, dim, i, j, step
    N, dim = data.shape[0], data.shape[1]
    cdef double sigma2, m, dist_sq_sum, vel_sq_sum, dt_over_m, hlp
    cdef double *force    = <double *>malloc(dim * sizeof(double))
    cdef double *velocity = <double *>malloc(dim * sizeof(double))
    cdef double *position = <double *>malloc(dim * sizeof(double))
    cdef double *tmp      = <double *>malloc(dim * sizeof(double))  
    trj = np.zeros(nrSteps*dim, dtype=np.float64)
    sig = np.zeros(nrSteps,     dtype=np.float64)
    sigma2    = sigma * sigma
    m         = mass / sigma2  # division by sigma for sigma-independent pitch
    dt_over_m = dt / m
    for i in range(dim):
        position[i] = initialpos[i]
        velocity[i] = initialvel[i]
    for step in range(nrSteps): 
        for i in range(dim): force[i]=0
        for j in range(N):
            dist_sq_sum = 0
            for i in range(dim):
                tmp[i] = position[i] - data[j,i]
                dist_sq_sum += tmp[i] * tmp[i]
            hlp = exp(-dist_sq_sum/sigma2)/sigma2
            for i in range(dim):
                force[i] += -tmp[i]*hlp
        # numerical integration => update pos and vel 
        vel_sq_sum = 0
        for i in range(dim):
            velocity[i] =  r * velocity[i] + force[i]*dt_over_m
            position[i] += dt * velocity[i]
            vel_sq_sum  += velocity[i] * velocity[i] 
        # store  
        sig[step] = vel_sq_sum
        offset    = step*dim
        for i in range(dim): 
            trj[offset+i] = position[i]
    # prepare return values
    trj     = np.reshape(trj, (-1, dim)) # correct array shape 
    lastpos = np.zeros(dim, dtype=np.float64)
    lastvel = np.zeros(dim, dtype=np.float64)
    for i in range(dim): 
        lastpos[i]=position[i]; 
        lastvel[i]=velocity[i]
    # free memory
    free(force); free(velocity); free(position); free(tmp)
    return trj, sig, lastpos, lastvel

In [6]:
def nneighbor(data, pos):
    """find index of nearest row vector in data to vector pos."""
    N, dim = np.shape(data)
    if(np.shape(pos)[0]<>dim): logger.warning("wrong dimension of pos"); return -1
    sqdists = sum(np.square(data - np.tile(pos, (N, 1))), 1)
    return np.argmin(sqdists)

## 5. Mode Explorer Example

### 5.1 Load test data

Data shape: 427 x 6

<img src="scatter_matric.png">

In [9]:
"""Load pregenerated data"""
data = np.load("test_data.npy")
fig = plt.figure()  # plot a side section
ax = fig.add_subplot(111)
ax.plot(data[:,0], data[:,1], '.')
N, dim  = np.shape(data)
s = np.std(data)  # std
data_dev = np.std(data)
sigmaBuf = 0.0

In [18]:
"""If you want to tryout some other randomly generated data, uncomment below 

arguments to set:
genNrMin, genNrMin (int) : min and max of number of data points per cluster
dim (int) : dimension
nc (int) : number of clusters
"""

# # Generate fake data
# genNrmin = 60
# genNrmax = 250
# dim = 5 # dimension
# nc = 3  # clusters
# # complex_data = DataGen().datagen(dim, nc, sigma=0.4, nr = nr) * 2
# data = DataGen().datagen(dim, nc, sigma=0.4, minnr=genNrmin, maxnr=genNrmax) * 2
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.plot(data[:,0], data[:,1], '.')
# N, dim  = np.shape(data)
# s = np.std(data)
# data_dev = np.std(data)
# sigmaBuf = 0.0
# print N, dim

'If you want to tryout some other randomly generated data, uncomment below \n\narguments to set:\ngenNrMin, genNrMin (int) : min and max of number of data points per cluster\ndim (int) : dimension\nnc (int) : number of clusters\n'

## 5.2 Scratch Mode

In [10]:
fig, axarr = plt.subplots(1,3, gridspec_kw = {'width_ratios':[5,3, 1]})
mngr = plt.get_current_fig_manager()
#mngr.window.setGeometry(0, 150, 1500, 800)
try:
    stopevent.set() 
except NameError:
    pass


# Setup Mode Explorer parameters: sigma, mass of particle, 
# r (resistence), dt (time step), nrSteps (step each movement)
sigma   = 0.6*s
mass    = 1.0 
r       = 0.999
dt      = 0.1
nrSteps = 256
sigmaBuf = sigma
lastpos = np.zeros(dim)
lastvel = np.zeros(dim)
maxamp = 1.
#--------------------------------------

# Set up plot
ix=0; iy=1
dsplt, = axarr[0].plot(data[:,ix], data[:,iy], "go", markersize=3)
trplt, = axarr[0].plot([], "r-", lw=5)
stplt, = axarr[0].plot([], "bo", markersize = 2)
sgplt, = axarr[1].plot([0,200],[0,2]  , "b-")
sigplt, = axarr[2].plot(0.5, sigma, 'bo', markersize=16)
axarr[2].axis([0, 1., 0, 2. * s])
plt.show(block=False)
fig.canvas.draw()
lock = False
#--------------------------------------

# Setup interactive plotting window to allow click and trigger. 
def on_click(event): 
    global lock; lock = True
    
def on_release(event): global lock; lock = False
    
def on_motion(event):
    global lock, sigma, mass, r, dt, nrSteps, sig, trj, lastpos, lastvel, sigmaBuf
    global lastTime, waitTime, maxamp
    if (lock):
        sigma = sigmaBuf
        pos2d = [event.xdata, event.ydata]
        lastposmf = data[nneighbor(data[:, [ix, iy]], pos2d), :]
        modeFound = False
        mfcnt = 0
        while(not modeFound and mfcnt<100):
            mfcnt +=1
            trjjunk, sigjunk, lastposmf, velJunk = PTSM(data, lastposmf, np.zeros(dim), sigma, mass, 0, 0.1, 100)
            if(np.linalg.norm(velJunk)<0.00001): modeFound=True
        modepot = potential(data, lastposmf, sigma)
        if(mfcnt<100):
            lastvel = np.random.rand(dim) * 0.03 * sigma
            lastpos = lastposmf
            trjjj, sigjj, lastposjj, lastveljj = PTSM(data, lastpos, lastvel, sigma, mass, 1, dt, 4*nrSteps)
            maxamp = max(sigjj)
        else:
            maxamp = 10000
    time.sleep(0.02)
    
def handle_close(evt):
    stopevent.set() 

# Connect handlers. 
fig.canvas.mpl_connect('close_event', handle_close)
cidclk = fig.canvas.mpl_connect('button_press_event', on_click)
cidrel = fig.canvas.mpl_connect('button_release_event', on_release)
cidmov = fig.canvas.mpl_connect('motion_notify_event', on_motion)
#--------------------------------------

def proc(fifo, stopevent):
    global pos, r, sigma, mass, dt, nrSteps, lastpos, lastvel, trj, sig, ix, iy, axarr , alpha
    global maxamp, sr
    sent = 0
    t0 = time.time()
    while not stopevent.wait(0):
        nrSteps = 100 + int(random.random()*200)
        sent = sent + nrSteps
        trj, sig, lastpos, lastvel = PTSM(data, lastpos, lastvel, sigma, mass, r, dt, nrSteps)
        if(max(sig)>5*maxamp): sig = sig * 0 
        fifo.put(sig.astype(np.float32) / maxamp * 0.2)  # send signal to audio pipeline
        dti = sent/sr - (time.time()-t0-0.15)
        if(dti>0.12):   
            trplt.set_data(trj[:,ix], trj[:,iy])
            sgplt.set_data(np.arange(np.shape(sig)[0]), sig / max(abs(sig)))
            sigplt.set_data(0.5, sigma)
            axarr[0].draw_artist(axarr[0].patch)
            axarr[0].draw_artist(dsplt)
            axarr[0].draw_artist(trplt)
            axarr[1].draw_artist(axarr[1].patch)
            axarr[1].draw_artist(sgplt)
            fig.canvas.draw()
            fig.canvas.flush_events()
        dti = sent/sr - (time.time()-t0-0.03)
        if(dti>0): time.sleep(dti)  
    logger.info("Mode Explorer Cleared.")
    
stopevent = threading.Event()
producer = threading.Thread(name="Compute audio signal", target=proc, args=[fifo, stopevent])
producer.start()

Mode Explorer Cleared.


## 5.3 Long decay mode, non-locking on interaction

In [39]:
"""Long decay method"""
fig, axarr = plt.subplots(1,3, gridspec_kw = {'width_ratios':[5,3, 1]})
mngr = plt.get_current_fig_manager()
mngr.window.setGeometry(0, 150, 1000, 700)
try:
    stopevent.set() 
except NameError:
    pass

# Setup ME parameters
sigma   = 0.6*s
r       = 0.9995
mass    = 1.0 
dt      = 0.1
nrSteps = 256
sigmaBuf = sigma
maxamp = 1.

# Setup graph
ix=0; iy=1
dsplt, = axarr[0].plot(data[:,ix], data[:,iy], "go", markersize=3)
trplt, = axarr[0].plot([], "r-", lw=1)
stplt, = axarr[0].plot([], "bo", markersize = 2)
sgplt, = axarr[1].plot([0,200],[0,2]  , "b-")
sigplt, = axarr[2].plot(0.5, sigma, 'bo', markersize=16)
axarr[2].axis([0, 1., 0, 2. * s])
plt.show(block=False)
fig.canvas.draw()
lastpos = np.zeros(dim)
lastvel = np.zeros(dim)
#--------------------------------------

def on_click(event): 
    global lastpos
    pos2d = [event.xdata, event.ydata]
    lastpos = data[nneighbor(data[:,[ix,iy]], pos2d),:]
def handle_close(evt):
    stopevent.set() 

fig.canvas.mpl_connect('close_event', handle_close)
cidclk = fig.canvas.mpl_connect('button_press_event', on_click)


def proc(fifo, stopevent):
    global pos, r, sigma, mass, dt, nrSteps, lastpos, lastvel, trj, sig, ix, iy, axarr , alpha
    global maxamp
    sent = 0
    srate = 11025.0
    t0 = time.time()
    while not stopevent.wait(0):
        nrSteps = 100 + int(random.random()*200)
        sent = sent + nrSteps
        sigmaBuf = sigma
        # print sigma
        trj, sig, lastpos, lastvel = PTSM(data, lastpos, lastvel, sigma, mass, r, dt, nrSteps)
        if(max(sig)>5*maxamp): sig = sig * 0 
        fifo.put(sig.astype(np.float32)/maxamp*0.2)
        dti = sent/srate - (time.time()-t0-0.15)
        if(dti>0.12):   
            trplt.set_data(trj[:,ix], trj[:,iy])
            sgplt.set_data(arange(shape(sig)[0]), sig/max(abs(sig)))
            sigplt.set_data(0.5, sigma)
            axarr[0].draw_artist(axarr[0].patch)
            axarr[0].draw_artist(dsplt)
            axarr[0].draw_artist(trplt)
            axarr[1].draw_artist(axarr[1].patch)
            axarr[1].draw_artist(sgplt)
            axarr[2].draw_artist(axarr[2].patch)
            axarr[2].draw_artist(sigplt)
            #axarr[1].draw_artist(sigplt)
            #axarr[2].draw_artist(sgplt)
            #sigplt, = axarr[2].plot(0.5, sigma, 'bo', markersize=16)
            #axarr[2].axis([0, 1., 0, 2. * sigma])
#             axarr[2].draw_artist(axarr[2].patch)
#             axarr[2].draw_artist(sigplt)
            fig.canvas.update()
            fig.canvas.flush_events()
        dti = sent/srate - (time.time()-t0-0.03)
        if(dti>0): time.sleep(dti)  
    logger.info("Mode Explorer Cleared.")
    
    
stopevent = threading.Event()
producer = threading.Thread(name="Compute audio signal", target=proc, args=[fifo, stopevent])
producer.start()

Mode Explorer Cleared.


In [35]:
"""Play around with the parameters"""
sigma   = 0.6*data_dev
r       = 0.99